In [23]:
!pip install pyspark

In [24]:
# Importações de bibliotecas principais
from pyspark.sql import SparkSession
from functools import reduce

# Importações de funções específicas do pyspark.sql.functions
from pyspark.sql.functions import (
    col, lit, lpad, mean, stddev, to_date, when, concat_ws
)


In [25]:
spark = SparkSession.builder.getOrCreate()

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
#dataframe
df = spark.read.csv('/content/drive/MyDrive/Projeto de Parceria  Semantix/br_ibge_ipca_mes_brasil.csv', header=True, inferSchema=True)

In [28]:
#Limpeza e pré-processamento dos dado

In [29]:
#visualizar dados
df.show(5)

+----+---+-----------+---------------+-------------------+------------------+--------------+-------------------+
| ano|mes|     indice|variacao_mensal|variacao_trimestral|variacao_semestral|variacao_anual|variacao_doze_meses|
+----+---+-----------+---------------+-------------------+------------------+--------------+-------------------+
|2000|  1|    1598.41|           0.62|               2.19|               4.3|          0.62|               8.85|
|2016|  1|    4550.23|           1.27|               3.27|              4.91|          1.27|              10.71|
|1985|  1|5.656761E-7|          11.76|              38.32|             86.66|         11.76|             221.27|
|1997|  1|    1379.33|           1.18|               1.98|              2.89|          1.18|               9.39|
|2014|  1|    3836.37|           0.55|               2.02|              3.21|          0.55|               5.59|
+----+---+-----------+---------------+-------------------+------------------+--------------+----

In [30]:
#tipo de dados
df.printSchema()

root
 |-- ano: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- indice: double (nullable = true)
 |-- variacao_mensal: double (nullable = true)
 |-- variacao_trimestral: double (nullable = true)
 |-- variacao_semestral: double (nullable = true)
 |-- variacao_anual: double (nullable = true)
 |-- variacao_doze_meses: double (nullable = true)



In [31]:
#volume de dados
df.count()

539

In [32]:

#Criar coluna de data ( Para ordenações e séries temporais)
df = df.withColumn(
    "data",
    to_date(
        concat_ws("-", col("ano"), lpad(col("mes"), 2, "0"), lit("01")),
        "yyyy-MM-dd"
    )
)


In [33]:
#contar dados nulos
from pyspark.sql.functions import col, sum, when

df.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in df.columns
]).show()


+---+---+------+---------------+-------------------+------------------+--------------+-------------------+----+
|ano|mes|indice|variacao_mensal|variacao_trimestral|variacao_semestral|variacao_anual|variacao_doze_meses|data|
+---+---+------+---------------+-------------------+------------------+--------------+-------------------+----+
|  0|  0|     0|              1|                  3|                 6|             1|                 12|   0|
+---+---+------+---------------+-------------------+------------------+--------------+-------------------+----+



In [34]:

#vizualizar dados nulos
linhas_com_nulos = df.filter(
    reduce(lambda a, b: a | b, [col(c).isNull() for c in df.columns])
)

linhas_com_nulos.show()

+----+---+----------+---------------+-------------------+------------------+--------------+-------------------+----------+
| ano|mes|    indice|variacao_mensal|variacao_trimestral|variacao_semestral|variacao_anual|variacao_doze_meses|      data|
+----+---+----------+---------------+-------------------+------------------+--------------+-------------------+----------+
|1980|  1| 8.1223E-9|           6.62|               NULL|              NULL|          6.62|               NULL|1980-01-01|
|1980|  2| 8.4973E-9|           4.62|               NULL|              NULL|         11.54|               NULL|1980-02-01|
|1980|  3| 9.0104E-9|           6.04|              18.27|              NULL|         18.27|               NULL|1980-03-01|
|1980|  4| 9.4867E-9|           5.29|               16.8|              NULL|         24.53|               NULL|1980-04-01|
|1980|  5|1.00277E-8|            5.7|              18.01|              NULL|         31.63|               NULL|1980-05-01|
|1980|  6|1.0559

In [35]:
# Filtrar datas a partir de 1981-01-01
df = df.filter(col("data") >= "1981-01-01")


In [36]:
linhas_com_nulos = df.filter(
    reduce(lambda a, b: a | b, [col(c).isNull() for c in df.columns])
)

linhas_com_nulos.show()

+---+---+------+---------------+-------------------+------------------+--------------+-------------------+----+
|ano|mes|indice|variacao_mensal|variacao_trimestral|variacao_semestral|variacao_anual|variacao_doze_meses|data|
+---+---+------+---------------+-------------------+------------------+--------------+-------------------+----+
+---+---+------+---------------+-------------------+------------------+--------------+-------------------+----+



In [37]:
# Verificar linhas duplicadas
linhas_duplicadas = df.groupBy(df.columns).count().filter(col("count") > 1)

# Mostrar as duplicadas, se existirem
linhas_duplicadas.show()


+---+---+------+---------------+-------------------+------------------+--------------+-------------------+----+-----+
|ano|mes|indice|variacao_mensal|variacao_trimestral|variacao_semestral|variacao_anual|variacao_doze_meses|data|count|
+---+---+------+---------------+-------------------+------------------+--------------+-------------------+----+-----+
+---+---+------+---------------+-------------------+------------------+--------------+-------------------+----+-----+



In [38]:
#periodo_economico: Categorização dos períodos econômicos brasileiros


df = df.withColumn(
    "periodo_economico",
    when(col("ano") <= 1986, "Pré-Plano Cruzado")
    .when((col("ano") >= 1987) & (col("ano") <= 1993), "Entre-Planos")
    .when((col("ano") >= 1994) & (col("ano") <= 1999), "Plano Real")
    .when((col("ano") >= 2000) & (col("ano") <= 2010), "Consolidação do Real")
    .when((col("ano") >= 2011) & (col("ano") <= 2014), "Nova Matriz Econômica")
    .when((col("ano") >= 2015) & (col("ano") <= 2016), "Crise Econômica")
    .when((col("ano") >= 2017) & (col("ano") <= 2019), "Recuperação e Reformas")
    .when((col("ano") >= 2020) & (col("ano") <= 2021), "Pandemia e Choques Globais")
    .when((col("ano") >= 2022) & (col("ano") <= 2024), "Inflação e Juros Altos")
    .otherwise("Fora da classificação")
)
df.select("ano", "periodo_economico").show()

+----+--------------------+
| ano|   periodo_economico|
+----+--------------------+
|2000|Consolidação do Real|
|2016|     Crise Econômica|
|1985|   Pré-Plano Cruzado|
|1997|          Plano Real|
|2014|Nova Matriz Econô...|
|1989|        Entre-Planos|
|2020|Pandemia e Choque...|
|2008|Consolidação do Real|
|2013|Nova Matriz Econô...|
|2024|Inflação e Juros ...|
|2012|Nova Matriz Econô...|
|2019|Recuperação e Ref...|
|1988|        Entre-Planos|
|1991|        Entre-Planos|
|2006|Consolidação do Real|
|2009|Consolidação do Real|
|2001|Consolidação do Real|
|1994|          Plano Real|
|1993|        Entre-Planos|
|1990|        Entre-Planos|
+----+--------------------+
only showing top 20 rows



In [39]:
#ciclo_economico: Classificação do momento econômico (expansão, contração, Estagnação/Contração.)

df = df.withColumn(
    "ciclo_economico",
    when(col("ano") <= 1986, "Contração")
    .when((col("ano") >= 1987) & (col("ano") <= 1993), "Contração")
    .when((col("ano") >= 1994) & (col("ano") <= 1999), "Expansão")
    .when((col("ano") >= 2000) & (col("ano") <= 2010), "Expansão")
    .when((col("ano") >= 2011) & (col("ano") <= 2014), "Contração")
    .when((col("ano") >= 2015) & (col("ano") <= 2016), "Contração")
    .when((col("ano") >= 2017) & (col("ano") <= 2019), "Expansão")
    .when((col("ano") >= 2020) & (col("ano") <= 2021), "Contração")
    .when((col("ano") >= 2022) & (col("ano") <= 2024), "Estagnação/Contração")
    .otherwise("Fora da classificação")
)

df.select("ano", "ciclo_economico").show()


+----+--------------------+
| ano|     ciclo_economico|
+----+--------------------+
|2000|            Expansão|
|2016|           Contração|
|1985|           Contração|
|1997|            Expansão|
|2014|           Contração|
|1989|           Contração|
|2020|           Contração|
|2008|            Expansão|
|2013|           Contração|
|2024|Estagnação/Contração|
|2012|           Contração|
|2019|            Expansão|
|1988|           Contração|
|1991|           Contração|
|2006|            Expansão|
|2009|            Expansão|
|2001|            Expansão|
|1994|            Expansão|
|1993|           Contração|
|1990|           Contração|
+----+--------------------+
only showing top 20 rows



In [40]:
#salvar arquivo

In [41]:
#DataFrame em um único CSV
df.coalesce(1) \
  .write \
  .option("header", True) \
  .mode("overwrite") \
  .csv("/content/drive/MyDrive/Projeto de Parceria  Semantix/Ipca_csv_temp")


In [42]:
#Mover e renomear o arquivo .csv
import os
import shutil

temp_path = "/content/drive/MyDrive/Projeto de Parceria  Semantix/Ipca_csv_temp"
final_csv_path = "/content/drive/MyDrive/Projeto de Parceria  Semantix/Ipca_final.csv"

# Encontrar o part-xxxxx.csv e mover
for file in os.listdir(temp_path):
    if file.startswith("part-") and file.endswith(".csv"):
        shutil.move(f"{temp_path}/{file}", final_csv_path)
        break


In [43]:
#Copiar o arquivo para o Colab para gerar o link de download:
shutil.copy(final_csv_path, "/content/Ipca_final.csv")


'/content/Ipca_final.csv'

In [44]:
from google.colab import files
files.download("/content/Ipca_final.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
spark.stop()